# All Imports

In [2]:
import os
from dotenv import load_dotenv
import requests
import time
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Semaphore

# Globals

In [21]:
DEFAULT_WR = 0.40
MAX_REQUESTS_PER_SECOND = 5
# Can use later for getting better avg/accounting for other gamemodes
EXCLUDED_QUEUE_IDS = {
    0,    # Custom games
    830,  # Co-op vs. AI: Intro bots
    840,  # Co-op vs. AI: Beginner bots
    850,  # Co-op vs. AI: Intermediate bots
    450,  # ARAM
    900,  # ARURF
    920,  # Nexus Blitz
    1300  # Nexus Blitz (old)
    # Add more queue IDs to exclude other game modes
}

# Getting Data

In [4]:
# Set up API (NA1 Region Only)
load_dotenv('.env')
api_key = os.getenv("RIOT_API_KEY")

In [5]:
# Testing key
#print(api_key)
#account_data = apiCallHandler(f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{summoner_name}/{summoner_tag}?api_key={api_key}')
#print(account_data)

RGAPI-95aec2f2-76d6-4d9b-b932-c842ba977c7b


In [6]:
# Custom RateLimiter class
class RateLimiter:
    def __init__(self, max_calls, period):
        self.max_calls = max_calls
        self.period = period
        self.calls = 0
        self.start_time = time.time()

    def acquire(self):
        current_time = time.time()
        elapsed = current_time - self.start_time
        if elapsed > self.period:
            self.calls = 0
            self.start_time = current_time
        if self.calls < self.max_calls:
            self.calls += 1
        else:
            time_to_wait = self.period - elapsed
            if time_to_wait > 0:
                time.sleep(time_to_wait)
            self.calls = 1
            self.start_time = time.time()

In [7]:
# Function for detecting and handling limit reached (429) and NOT success (200)
def apiCallHandler(request_url):
    #print(request_url)
    response = requests.get(request_url)

    while response.status_code != 200: # while loop here for later when we want to ignore error 429
        if(response.status_code == 429):
            # Not success but is 429 API limit error
            print("TMP: status 429 detected")
            sys.exit("Stopping all execution")
        else:
            # Not success and not 429 API limit error
            print(f"Failed to fetch data: {response.status_code}")
            sys.exit("Stopping all execution")
    # (finally) status of 200
    return response.json()

In [11]:
# Function for multithread api calls
def apiMultiCallHandler(request_url, rate_limiter):
    rate_limiter.acquire()
    response = requests.get(request_url)
    while response.status_code != 200: # while loop here for later when we want to ignore error 429
        if(response.status_code == 429):
            # Not success but is 429 API limit error
            print("TMP: status 429 detected")
            sys.exit("Stopping all execution")
        else:
            # Not success and not 429 API limit error
            print(f"Failed to fetch data: {response.status_code}")
            sys.exit("Stopping all execution")
    # (finally) status of 200
    return response.json()

In [12]:
# Function for using multithreading when calling multiple APIs
# urls = a list of urls desired (and compatible) to multithread
def multithread_call(urls):
    results = []
    # Rate_limiters for limiting requests, threadpoolexecutor ensures max concurrent workers don't exceed
    rate_limiter = RateLimiter(max_calls=MAX_REQUESTS_PER_SECOND, period=1)
    with ThreadPoolExecutor(max_workers=MAX_REQUESTS_PER_SECOND) as executor:
        # "Future" objects store the future value of the API call
        futures = [executor.submit(apiMultiCallHandler, f"https://americas.api.riotgames.com/lol/match/v5/matches/{url}?api_key={api_key}", rate_limiter) for url in urls]
        # as_completed takes "Future" objects in the order they complete
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                # This shouldn't ever happen, API error checking done in the API handler
                print(f"Exception: {e}")
                sys.exit("Stopping all execution")
    return results

In [13]:
# Function for calculating average win rate of a summoner using multithreading
def avg_wr_summoner(match_history, summoner_puuid):
    if(len(match_history) == 0):
        return DEFAULT_WR
    results = multithread_call(match_history)
    total_matches = 0
    win_count = 0
    for response in results:
        try:
            # Finds the first participant's id who's equal to the summoner's puuid
            participant = next(p for p in response['info']['participants'] if p['puuid'] == summoner_puuid)
            if(participant['win'] == True):
                win_count+=1
            total_matches+=1
        except StopIteration:
            print("ERROR: StopIteration exception occured, Riot data incorrect?")
            sys.exit("Stopping all execution")
        except:
            print("ERROR: Exception occured, shouldn't be here")
            sys.exit("Stopping all execution")
    return round(win_count/total_matches, 2)

In [23]:
# Gets Data for a given sumoner name and summoner tag
# Data currently equals: Summoner Level, AVG win rate for past 20 matches, 
# def get_something

# tmp, replace with ID of summoner
summoner_name = "kneesyd"#"portal"
summoner_tag = "NA1"#"only2"

# Get PUUID based on name & tag; max api_calls = 1 (No multithread)
account_data = apiCallHandler(f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{summoner_name}/{summoner_tag}?api_key={api_key}')
print(f"Summoner ID: {summoner_name}#{summoner_tag}")
print(f"Summoner PUUID: {account_data['puuid']}")
summoner_puuid = account_data['puuid']

# Get summoners' match history & length using their PUUID, filters only for normal draft games; max api_calls = 1 (No multithread)
summoner_match_history = apiCallHandler(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{summoner_puuid}/ids?queue=400&type=normal&start=0&count=20&api_key={api_key}')
summoner_match_history_length = len(summoner_match_history)

# Get summoners' level using their PUUID; max api_calls = 1
summoner_data = apiCallHandler(f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{summoner_puuid}?api_key={api_key}')
summoner_level = summoner_data['summonerLevel']

# Get summoners' win rate using match history; max_api_calls = 20 
# Future work: Ensure that no custom, bot, tutorial, or arena/limited game mode matches
# Future work: If match count is < 20, THEN use quickplay and aram stats (assign weights to them)
# Future work: add compatibility for predicting specifically gamemodes other than normal draft 
avg_wr = avg_wr_summoner(summoner_match_history, summoner_puuid)


Summoner ID: kneesyd#NA1
Summoner PUUID: mO4gr-whHun9IRseQslUEGJlORnH1SO3x-uUcAWVjsqyI8GKIQ5oysRDMqRHXSmbG6uzE3m4ZzoO8Q


# Placing Data Into CSV File

# Loading Data